🖖🏼 Story of my life so far ...

An autobiographical story-form graph visualization of my experience. The semantics are in the ➡️ relationships, as these are how we ascribe meaning to our lives. The magic happens in the transitions ➡️ between where we are and where we intend to go. 

Authored in markdown, video, JSON, python, and SVG.

In [17]:
import pandas as pd

# pd.read_json("sample.json",lines=True,orient='columns')
df = pd.read_json('data.json')
# [[{Value1:1},{value2:2}],[{value3:3},{value4:4}],.....]
# for data like so use
# value1 = df['column_name'][0][0].get(Value1)
newdf = df.drop("uri",axis='columns')

print(newdf)

df.head()

  provenance  qualified                bonus
0    youtube       True                  NaN
1     github       True                  NaN
2   nlm umls      False  {'name': 'library'}


,provenance,uri,qualified,bonus
0,youtube,youtube,True,NaN
1,github,github,True,NaN
2,nlm umls,nlm umls,False,{'name': 'library'}
